In [1]:
import sys, os, json, random
from collections import *

In [2]:
ls

collapse_silence.ipynb
compare_gold_alignments.ipynb
compare_gold_readings.py*
f0s.csv
item_to_reading.json
item_to_session_to_alignment.json
item_to_session_to_alignment_collapsed_pauses.json
recs.csv
recwords.csv
serialize_forced_alignments.ipynb*
xTrans.csv


In [3]:
def collapse_alignment(align):
    idx = 0
    current_items = []
    condensed_align = []
    for item in align:
        curr_token, sframe, nframes = item
        if '<' not in curr_token:
            if current_items:
                if len(current_items) == 1:
                    current_items = current_items[0]
                condensed_align.append(current_items)
            condensed_align.append(item)
            current_items = []
        else:
            current_items.append(item)
    if current_items:
        if len(current_items) == 1:
            current_items = current_items[0]
        condensed_align.append(current_items)
    for i in range(len(condensed_align)):
        if type(condensed_align[i][0]) == type([]):
            first_sframe = condensed_align[i][0][1]
            total_nframes = sum(x[2] for x in condensed_align[i])
            condensed_align[i] = ['<pause>', first_sframe, total_nframes]
    for i, elem in enumerate(condensed_align):
        if '<' not in elem[0]:
            continue
        condensed_align[i] = ['<pause>', elem[1], elem[2]]
    return condensed_align

In [4]:
with open('item_to_session_to_alignment.json') as f:
    item_to_session_to_alignment = json.load(f)

In [5]:
condensed_item_to_session_to_alignment = defaultdict(lambda: defaultdict(list))
for item, session_to_alignment in item_to_session_to_alignment.items():
    for session, alignments in session_to_alignment.items():
        condensed = collapse_alignment(alignments)
        condensed_item_to_session_to_alignment[item][session] = condensed

In [6]:
condensed_dict = {k: dict(v) for k, v in condensed_item_to_session_to_alignment.items()}

In [7]:
# with open('item_to_session_to_alignment_collapsed_pauses.json', 'x') as f:
#     json.dump(condensed_dict, f, indent=2)